In [1]:
import minna_functions as mf

In [2]:
get_playlist = mf.get_spotify_playlist('BQD-YFHHfR4bgdzxHuqYiB36E1OqyMGc9w2GuJSjItIymJPn0D2rmCujOxrkq7l6f5oahRE0lDNo1KjWuIhAQoZSBjZYp7Ugixokb3uou4hOwoP4nNPD26bLCFG85idTtKSVHdm2eG_zz6CKbI0')

In [3]:
get_playlist.get_playlist_SQL('4JgPn5zouXilXoM2v1Istn', 'test')

In [4]:
df = get_playlist.SQL_to_dataframe('test')
df.head()

,song_name,song_id,popularity,artist_name,speechiness,danceability
index,,,,,,
1,bad guy,2Fxmhks0bxGSBdJ92vM42m,98,Billie Eilish,None,None
2,Goodbyes (Feat. Young Thug),6vBdBCoOhKHiYDDOcorfNo,89,Post Malone,None,None
3,Señorita,0TK2YIli7K1leLovkQiNik,100,Shawn Mendes,None,None
4,Money In The Grave (Drake ft. Rick Ross),5ry2OE6R2zPQFDO85XkgRb,93,Drake,None,None
5,Panini,6fTt0CH2t0mdeB2N9XFG5r,91,Lil Nas X,None,None


In [5]:
df2 = get_playlist.song_audio_features(df, 'test')

None


In [6]:
df2.head()

,song_name,song_id,popularity,artist_name,speechiness,danceability
index,,,,,,
1,bad guy,2Fxmhks0bxGSBdJ92vM42m,98,Billie Eilish,0.3750,0.701
2,Goodbyes (Feat. Young Thug),6vBdBCoOhKHiYDDOcorfNo,89,Post Malone,0.0809,0.593
3,Señorita,0TK2YIli7K1leLovkQiNik,100,Shawn Mendes,0.0287,0.759
4,Money In The Grave (Drake ft. Rick Ross),5ry2OE6R2zPQFDO85XkgRb,93,Drake,0.0460,0.831
5,Panini,6fTt0CH2t0mdeB2N9XFG5r,91,Lil Nas X,0.0752,0.703


In [6]:
import sqlite3

In [7]:
conn = sqlite3.connect('spotify.db')

In [8]:
c = conn.cursor()

In [9]:
c.execute('''DROP TABLE test''')